# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Benisty  ->  M. Benisty  |  ['M. Benisty']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
T. Müller  ->  T. Müller  |  ['T. Müller']
N. Hoyer  ->  N. Hoyer  |  ['N. Hoyer']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']


H. Beuther  ->  H. Beuther  |  ['H. Beuther']
S. Bosman  ->  S. Bosman  |  ['S. Bosman']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
S. Brown  ->  S. Brown  |  ['S. Brown']
Arxiv has 95 new papers today
          10 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2512.14782


extracting tarball to tmp_2512.14782... done.


M. Benisty  ->  M. Benisty  |  ['M. Benisty']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:745: LatexWarning: Could not extract abstract from tmp_2512.14782/main.tex
  warnings.warn(LatexWarning(f"Could not extract abstract from {self.main_file}"))


...

 done.


Retrieving document from  https://arxiv.org/e-print/2512.15652
Retrieving document from  https://arxiv.org/e-print/2512.15696


/tmp/ipykernel_2878/2822249172.py:52: LatexWarning: 2512.15652 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2512.15696...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.14782-b31b1b.svg)](https://arxiv.org/abs/2512.14782) | **Why the Northern Hemisphere Needs a 30-40 m Telescope and the Science at Stake. How do Planetary Systems Form?**  |
|| I. Mendigutía, et al. -- incl., <mark>M. Benisty</mark>, <mark>G. Chauvin</mark>, <mark>R. v. Boekel</mark> |
|*Appeared on*| *2025-12-18*|
|*Comments*| *White Paper submitted to "ESO Expanding Horizons: Transforming Astronomy in the 2040s"*|
|**Abstract**|            The detection and characterization of protoplanets in protoplanetary disks around young stars is emerging as a transformative field that will redefine our understanding of how planetary systems form. While current facilities have revealed the diversity of mature exoplanets and the complex structures of disks, we still lack the crucial observational link between them: a statistically meaningful census of planets caught in the act of formation. This white paper argues that such a breakthrough requires access to the nearest and most informative star-forming regions, roughly half of which are poorly accessible or entirely unreachable from Cerro Armazones. Although the ELT alone will strongly impact our knowledge of planet formation, its location prevents Europe from fully exploiting the necessary parameter space. A 30-40 m telescope in the northern hemisphere is therefore essential for obtaining diffraction-limited imaging of protoplanets and disks across the entire sky, enabling robust demographics, exploiting synergies with ngVLA, Gaia, and other facilities covering the north, and ensuring that Europe remains at the forefront of the planet-formation revolution in the coming decades.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.15054-b31b1b.svg)](https://arxiv.org/abs/2512.15054) | **A global view on star formation: The GLOSTAR Galactic plane survey XII. Effelsberg's continuum view and data release**  |
|| Y. Gong, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2025-12-18*|
|*Comments*| *19 pages, 18 figures, accepted for publication in A&A*|
|**Abstract**|            Extended radio continuum emission and its linear polarization play a key role in probing large-scale structures of synchrotron and free-free emission in the Milky Way. Despite the existence of many radio continuum surveys, sensitive and high-angular-resolution single-dish surveys of extended radio continuum emission remain scarce. As part of the GLObal view of STAR formation (GLOSTAR) survey, we will also crucially complement the existing data from the Karl G. Jansky Very Large Array (VLA) by addressing the missing zero-spacing gap. Within the framework of the GLOSTAR Galactic plane survey, we performed large-scale radio continuum imaging observations toward the Galactic plane in the range $-2^{\circ}< \ell <60^{\circ}$ and $|b|<1.1^{\circ}$, as well as the Cygnus X region with the Effelsberg 100-m radio telescope. We present the Effelsberg continuum survey at 4.89GHz and 6.82GHz including linear polarization with angular resolutions of 145$^{\prime\prime}$ and 106$^{\prime\prime}$, respectively. The survey has been corrected for missing large-scale emission using available low-angular-resolution surveys. Comparison with previous single-dish surveys indicates that our continuum survey represents the highest-quality single-dish data collected to date at this frequency. More than 90\% of the flux density missed by the VLA D-array data is effectively recovered by the Effelsberg continuum survey. The improved sensitivity and angular resolution of our survey enable reliable mapping of Galactic magnetic field structures, with polarization data that are less affected by depolarization than in previous surveys. The GLOSTAR single-dish continuum data will be released publicly, offering a valuable resource for studying extended objects including HII regions, supernova remnants, diffuse interstellar medium, and Galactic structure. (Abridged)         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.15127-b31b1b.svg)](https://arxiv.org/abs/2512.15127) | **Stellar Morphology of Optically Dark or Faint Galaxies at $z>3$ with**  |
|| A. Ganguly, et al. -- incl., <mark>S. Bosman</mark> |
|*Appeared on*| *2025-12-18*|
|*Comments*| **|
|**Abstract**|            JWST offers an unprecedented view of optically dark or faint galaxies (OFGs), previously missed by HST. They are likely massive, heavily dust-obscured star-forming galaxies (SFGs) that substantially contribute to the cosmic SFR density at z > 3. To identify drivers of their high dust attenuation and their role in early universe galaxy evolution, we analyse the stellar morphology of 65 OFGs (from 1892 SFGs at 3<z<4) using NIRCAM/F444W imaging from the PRIMER and CEERS fields. We study correlations between dust attenuation ($A_{v}$) and galaxy properties, like stellar mass, size, and orientation, and compare scaling relations between OFGs and typical SFGs. We find that OFGs are ~8-9 times more massive and ~4 times more dust attenuated than the parent sample. Structurally, OFGs resemble parent SFGs in median $R_e$ and median $\Sigma_{R_e}$ but may be slightly rounder on average. While $A_v$ strongly correlates with stellar mass, it does not show significant dependence on stellar mass-normalised effective radius and stellar mass surface density, Sérsic index, axis ratio, or SFR surface density. The mass-size and mass-surface density relations place OFGs as a higher-mass extension of SFGs, with no concrete proof of evolutionary differences between them. This suggests that OFGs are heavily dust-obscured primarily due to their high stellar masses, which facilitates dust production and retention, with older stellar populations likely contributing as well. Although some OFGs exhibit high $\Sigma_\mathrm{R_e}$ and occupy regions of the mass-size plane similar to quiescent galaxies, the overall sample is not representative of this. Their current structures resemble typical SFGs, with no concrete signs of rapid compaction. Diversity in their physical properties shows that OFGs span a range of evolutionary states with few showing reduced star formation, while most remain actively star-forming.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.15637-b31b1b.svg)](https://arxiv.org/abs/2512.15637) | **The impact of bars on the properties of HII regions in the TIMER survey**  |
|| L. Sánchez-Menguiano, et al. -- incl., <mark>J. Neumann</mark> |
|*Appeared on*| *2025-12-18*|
|*Comments*| *Published in MNRAS*|
|**Abstract**|            In this study we perform a comparative analysis of the properties of the HII regions located in different areas of barred galaxies, with the aim of investigating the impact of bars on the physical properties of the ionised gas. Based on integral field spectroscopy data for 17 barred galaxies covering approximately the central 6x6 kpc, we detect a total of 2200 HII regions, of which 331 are located within the nuclear disc (also known as circumnuclear regions), 661 in the bar region, and 1208 in the disc. Among the physical properties of the HII regions, we explore the O/H and N/O abundances, H$\alpha$ luminosity, dust extinction, electron density, and H$\alpha$ equivalent width. We find clear differences in the properties of the HII regions between the nuclear disc, the bar and the disc, that could be explained by an enhancement in the molecular gas concentration in the central parts driven by bar-induced gas flows. As this gas is channelled towards the galaxy centre, the most extreme values in the analysed properties are found for the circumnuclear HII regions. Unlike the bar strength, galaxy mass does seem to affect the properties of the HII regions, with massive galaxies presenting higher values in most of the properties, possibly due to the increased amount of gas in these systems. This study provides evidence that the bar-driven redistribution of material within the galaxy inner parts causes significant differences in the HII region properties depending on their location within the galaxies.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.14786-b31b1b.svg)](https://arxiv.org/abs/2512.14786) | **Why the Northern Hemisphere Needs a 30-40 m Telescope and the Science at Stake: Mapping formation pathways of nuclear star clusters across galaxies**  |
|| F. Pinna, et al. -- incl., <mark>N. Hoyer</mark> |
|*Appeared on*| *2025-12-18*|
|*Comments*| *White Paper submitted to "ESO Expanding Horizons: Transforming Astronomy in the 2040s"*|
|**Abstract**|            Nuclear star clusters (NSCs) are dense, compact stellar systems only a few parsecs across, located at galaxy centers. Their small sizes make them difficult to resolve spatially. NSCs often coexist with massive black holes, and both trace the dynamical state and evolution of their host galaxies. Dense stellar environments such as NSCs are also ideal sites for forming intermediate-mass black holes (IMBHs). To date, spatially resolved NSC properties, crucial for reconstructing dynamical and star-formation histories, have only been obtained for galaxies within 5 Mpc, using the highest-resolution instruments on the current class of very large telescopes. This severely limits spectroscopic studies, and a systematic, unbiased survey has never been accomplished. Because the vast majority of known NSCs are located in the Northern Hemisphere, only a 30-m-class telescope in the North can provide the statistical power needed to study their physical properties and measure the mass of coexisting central black holes. We propose leveraging the capabilities of a 30-m-class Northern telescope to obtain the first comprehensive, spatially resolved survey of NSCs, finally allowing us to unveil their formation pathways and their yet unknown connection with central massive black holes.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.15159-b31b1b.svg)](https://arxiv.org/abs/2512.15159) | **Discussion on the vanishment of solar atmospheric structures during magnetic reconnection**  |
|| J. Zhang, T. Ding, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-12-18*|
|*Comments*| *9 pages, 4 figures, 3 attached movies and 43 references*|
|**Abstract**|            In solar atmosphere, magnetic reconnection alters the topological connectivity, and magnetic energy is released. However, the length change of the reconnecting structures has rarely been reported. To identify the evolution of the topological structures, we search for reconnection events which should satisfy 3 criteria. (1) Each event displays an explicit X-type configuration, and the configuration consists of two sets of independent atmospheric structures, (2) the reconnection process is clearly observed, and (3) the topological connectivity of the structures can be tracked from at least 5 minutes prior to the occurrence of magnetic reconnection to 5 minutes after the reconnection. In this work, 3 events are selected and studied. During the reconnection moment, the total length of the two topological structures in each event shortens suddenly, and the decrements for events 1--3 are 47 Mm, 3.7 Mm, and 8.2 Mm, respectively, implying that partial structures vanish observationally during magnetic reconnection process. Several possibilities about the vanishment, e.g. the shrinkage of the reconnecting structures due to magnetic tension, the bizarre change in the third dimension, and magnetic field annihilation, have been discussed.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.15696-b31b1b.svg)](https://arxiv.org/abs/2512.15696) | **Structure of Jupiter's High-Latitude Storms: Folded Filamentary Regions Revealed by Juno**  |
|| L. Fletcher, et al. -- incl., <mark>S. Brown</mark> |
|*Appeared on*| *2025-12-18*|
|*Comments*| *53 pages, 10 figures, accepted for publication in JGR:Planets*|
|**Abstract**|            Sprawling, turbulent cloud formations dominate the meteorology of Jupiter's mid-to-high latitudes, known as Folded Filamentary Regions (FFRs). A multi-wavelength characterisation by Juno reveals the spatial distribution, vertical structure, and energetics of the FFRs. The cloud tops display multiple lobes of stratiform aerosols, separated by darker, cloud-free lanes, and embedded with smaller eddies and high-altitude cumulus clouds. These cyclonic FFRs are microwave-bright in shallow-sounding wavelengths ($p<5$ bars) and microwave-dark in deep-sounding wavelengths ($p>10$ bars), with the transition potentially associated with the water condensation layer (6-7 bars). Associating microwave contrasts with temperature anomalies, this implies despinning of cyclonic eddies above/below their mid-planes. Despite deep roots (being detectable in wavelengths sounding $\sim100$ bars), they are ``pancake vortices'' with horizontal extents at least an order of magnitude larger than their depth. In the northern hemisphere, FFRs are most common in cyclonic belts poleward of $40^\circ$N (all latitudes are planetocentric), particularly a North Polar Filamentary Belt (NPFB) near $66-70^\circ$N that defines the transition from organised belts/zones to the chaotic polar domain. This distribution explains the high lightning rates from $45-80^\circ$N, peaking in a belt poleward of $52.3^\circ$N, which may trace the availability of water for moist convection. Many observed lightning flashes can be associated to specific FFRs containing bright storms, but some FFRs display no activity, suggesting quiescent periods during storm evolution. Analogies to Earth's oceanic eddies suggest that cyclones deform isentropic surfaces at their midplanes, raising deep water-rich layers upwards to promote moist convection, release latent heat, and inject clouds into the upper troposphere.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.14790-b31b1b.svg)](https://arxiv.org/abs/2512.14790) | **Why the Northern Hemisphere Needs a 30-40 m Telescope and the Science at Stake: Northern Local Star-forming Dwarf Galaxies. Analogues of the First Galaxies and Probes of the Cosmic Metallicity Scale**  |
|| C. Esteban, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2025-12-18*|
|*Comments*| *White Paper submitted to "ESO Expanding Horizons: Transforming Astronomy in the 2040s"*|
|**Abstract**|            Star-forming dwarf galaxies in the local Universe, especially extremely metal-poor ones, can be considered analogous to early galaxies of the Epoch of Reionization (z >= 6). Currently available telescopes cannot adequately detect and measure heavy element recombination lines and certain faint collisionally excited lines, which are essential for exploring the effects and biases that potential inhomogeneities in electron temperature and density of the ionized gas may have on determining the chemical composition of these galaxies. On the other hand, the origin of very high-ionization lines (e.g. He II, [Ne V], C IV]) measured in the spectra of an important fraction of these objects remains unknown and a challenge to current stellar models, suggesting the presence of Population III-like stars and/or the existence of non-conventional ionizing sources. Obtaining very deep spectra for a selected sample of local star-forming dwarf galaxies would provide unprecedented constraints on their nature, ionization and true chemical abundances, and could change the metallicity scale we assume to understand the chemical evolution of galaxies over cosmic time.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.15652-b31b1b.svg)](https://arxiv.org/abs/2512.15652) | **Towards ALMA2040: An update from the European community and invitation to contribute**  |
|| S. Facchini, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-12-18*|
|*Comments*| *Community report and status update for the ALMA2040 Expanding Horizons initiative; 8 pages*|
|**Abstract**|            Over the last 15 years, the Atacama Large Millimeter/submillimeter Array (ALMA) has revolutionized astrophysics by providing unprecedented resolution and sensitivity in observing the cold universe, including the formation of stars, planets, and galaxies. With groundbreaking discoveries ranging from the first detailed images of protoplanetary disks to the kinematics of galaxies in the Epoch of Reionization, ALMA has showcased the vast discovery potential of the (sub-)mm wavelength regime. However, in another 15 years from now--in the 2040s--the science landscape will have changed dramatically as new major observational facilities will have started their operations or have come towards advanced maturity in their scientific outcome (e.g., JWST, Rubin Observatory, ELT, Euclid, Gaia, Plato, Ariel, Roman Space Telescope, SPHEREx, LiteBIRD, LISA, SKA and others). At the same time, ALMA's current Wideband Sensitivity Upgrade will have been in place for ~10 years, and ALMA itself will have been operational for 30 years. To fully exploit this era, the community needs a next-generation facility operating at (sub-)mm wavelengths with capabilities far beyond those possible within ALMA's current infrastructure. To this end, ALMA2040 is a community-driven initiative to define the key scientific questions of the 2040s and translate them into a technical vision for a next-generation transformational (sub-)millimeter facility. Our goal with this document is to summarize the current status of the effort, synthesize outcomes from the 2025 workshops, outline next steps toward a reference design concept, and invite broad participation from the global mm/sub-mm community to help shape this future facility. In the following we provide details on the process and scope. We invite everyone who wishes to join the effort and/or contribute to the dedicated White Papers planned for 2026.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2512.14782/./obs_exoplanets_SFRs_horizontal.png']
copying  tmp_2512.14782/./obs_exoplanets_SFRs_horizontal.png to _build/html/
exported in  _build/html/2512.14782.md
    + _build/html/tmp_2512.14782/./obs_exoplanets_SFRs_horizontal.png
found figures ['tmp_2512.15054/./fig/GLOSTAR-4.88GHz-I.png', 'tmp_2512.15054/./fig/GLOSTAR-4.88GHz-I.png', 'tmp_2512.15054/./fig/Parkes-Urumqi.png']
copying  tmp_2512.15054/./fig/GLOSTAR-4.88GHz-I.png to _build/html/
copying  tmp_2512.15054/./fig/GLOSTAR-4.88GHz-I.png to _build/html/
copying  tmp_2512.15054/./fig/Parkes-Urumqi.png to _build/html/
exported in  _build/html/2512.15054.md
    + _build/html/tmp_2512.15054/./fig/GLOSTAR-4.88GHz-I.png
    + _build/html/tmp_2512.15054/./fig/GLOSTAR-4.88GHz-I.png
    + _build/html/tmp_2512.15054/./fig/Parkes-Urumqi.png
found figures ['tmp_2512.15127/./images/av_properties.png', 'tmp_2512.15127/./images/sigma-re.png', 'tmp_2512.15127/./mass-size.png', 'tmp_2512.15127/./images/mass_size_black.pn

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\bibfont}{\footnotesize}$</div>



<div id="title">

# Why the Northern Hemisphere Needs a 30--40 m Telescope\ and the Science at Stake.\ How do Planetary Systems Form?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.14782-b31b1b.svg)](https://arxiv.org/abs/2512.14782)<mark>Appeared on: 2025-12-18</mark> -  _White Paper submitted to "ESO Expanding Horizons: Transforming Astronomy in the 2040s"_

</div>
<div id="authors">

I. Mendigutía\thanks{\texttt{imendigutia@cab.inta-csic.es}}, et al. -- incl., <mark>M. Benisty</mark>, <mark>G. Chauvin</mark>, <mark>R. v. Boekel</mark>

</div>
<div id="abstract">

**Abstract:**            The detection and characterization of protoplanets in protoplanetary disks around young stars is emerging as a transformative field that will redefine our understanding of how planetary systems form. While current facilities have revealed the diversity of mature exoplanets and the complex structures of disks, we still lack the crucial observational link between them: a statistically meaningful census of planets caught in the act of formation. This white paper argues that such a breakthrough requires access to the nearest and most informative star-forming regions, roughly half of which are poorly accessible or entirely unreachable from Cerro Armazones. Although the ELT alone will strongly impact our knowledge of planet formation, its location prevents Europe from fully exploiting the necessary parameter space. A 30-40 m telescope in the northern hemisphere is therefore essential for obtaining diffraction-limited imaging of protoplanets and disks across the entire sky, enabling robust demographics, exploiting synergies with ngVLA, Gaia, and other facilities covering the north, and ensuring that Europe remains at the forefront of the planet-formation revolution in the coming decades.         

</div>

<div id="div_fig1">

<img src="tmp_2512.14782/./obs_exoplanets_SFRs_horizontal.png" alt="Fig1" width="100%"/>

**Figure 1. -** \small**(Left)** Exoplanet discoveries through the years (from the Encyclopaedia of exoplanetary systems; \href{https://exoplanet.eu}{https://exoplanet.eu/}). **(Right)** Galactic coordinates of the main star forming regions. The closest ones partially or totally located at distances $\leq$ 300 pc are indicated with capital letters. **(Both)** The colour code indicates whether the sources can be observed only from the South (declination $\delta$$\lesssim$ -31$^{\circ}$), only from the North ($\delta$$\gtrsim$ 35$^{\circ}$), or from both hemispheres. The Cerro Armazones and La Palma observatories are taken as reference, assuming that a source cannot be observed for elevations $<$ 30$^{\circ}$. Roughly half of the exoplanets and protoplanetary disks can be better or only covered from the North. (*fig:exoplanet_and_SFR_coverage*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.14782"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Teff}{T_{\rm{eff}}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\Wmq}{Wm^{-2}}$
$\newcommand{\ergps}{erg s^{-1}}$
$\newcommand{\mum}{ \mum}$
$\newcommand{◦ee}{^{\circ}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Lsun}{L_{\odot}}$
$\newcommand{\Msunyr}{ M_{\odot}yr^{-1}}$
$\newcommand{\OI}{[O I]}$
$\newcommand{\SII}{[S II]}$
$\newcommand{\NII}{[N II]}$
$\newcommand{\CaII}{Ca II}$
$\newcommand{\HeI}{He I}$
$\newcommand{\LiI}{Li I}$
$\newcommand{\nodata}{...}$
$\newcommand{\accunit}{M_{\odot} yr^{-1}}$
$\newcommand{\rev}{ }$
$\newcommand{\newrev}{ }$
$\newcommand{\newnewrev}{\bf }$
$\newcommand{\fig}{Fig.}$
$\newcommand{\Mjup}{M_{\rm{Jup}}}$
$\newcommand{\Mdotacc}{\dot M_{\rm{acc}}}$
$\newcommand{\gisela}[1]{{\bf \color{black}[#1]}}$</div>



<div id="title">

# A global view on star formation: The GLOSTAR Galactic plane survey: XII. Effelsberg's continuum view and data release

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.15054-b31b1b.svg)](https://arxiv.org/abs/2512.15054)<mark>Appeared on: 2025-12-18</mark> -  _19 pages, 18 figures, accepted for publication in A&A_

</div>
<div id="authors">

Y. Gong, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** Extended radio continuum emission and its linear polarization play a key role in probing large-scale structures of synchrotron and free-free emission in the Milky Way. Despite the existence of many radio continuum surveys, sensitive and high-angular-resolution single-dish surveys of extended radio continuum emission remain scarce. Our objective is to deliver a Galactic plane survey of extended radio continuum emission within the frequency range of 4--8 GHz, achieving an unprecedented angular resolution of $\lesssim$ 3 $\arcmin$ . As part of the GLObal view of STAR formation (GLOSTAR) survey, we will also crucially complement the existing data from the Karl G. Jansky Very Large Array (VLA) by addressing the missing zero-spacing gap. Within the framework of the GLOSTAR Galactic plane survey, we performed large-scale radio continuum imaging observations toward the Galactic plane in the range $-2^{\circ}< \ell <60^{\circ}$ and $|b|<1.1^{\circ}$ , as well as the Cygnus X region $76^{\circ}< \ell <83^{\circ}$ and $-1^{\circ}<b<2^{\circ}$ with the Effelsberg 100-m radio telescope. We present the Effelsberg continuum survey at 4.89 GHz and 6.82 GHz including linear polarization with angular resolutions of 145 $\arcsec$ and 106 $\arcsec$ , respectively. The survey has been corrected for missing large-scale emission using available low-angular-resolution surveys. Comparison with previous single-dish surveys indicates that our continuum survey represents the highest-quality single-dish data collected to date at this frequency. More than 90 \% of the flux density missed by the VLA D-array data is effectively recovered by the Effelsberg continuum survey. The improved sensitivity and angular resolution of our survey enable reliable mapping of Galactic magnetic field structures, with polarization data that are less affected by depolarization than in previous surveys. The GLOSTAR single-dish continuum data will be released publicly, offering a valuable resource for studying extended objects including H ${\scriptsize II}$ regions, supernova remnants, diffuse interstellar medium, and Galactic structure.

</div>

<div id="div_fig1">

<img src="tmp_2512.15054/./fig/GLOSTAR-4.88GHz-I.png" alt="Fig6" width="100%"/>

**Figure 6. -** {Effelsberg 4.89 GHz Stokes I continuum map of the whole GLOSTAR survey.} (*Fig:4.89GHz-I*)

</div>
<div id="div_fig2">

<img src="tmp_2512.15054/./fig/GLOSTAR-4.88GHz-I.png" alt="Fig8" width="100%"/>

**Figure 8. -** {Effelsberg 4.89 GHz Stokes $I$ continuum map of the whole GLOSTAR survey. Zoom-in plots of each fields are available via \url{https://gongyan2444.github.io/glostar-snr-hii.html}, where the green, gray, and red circles represent the known SNRs \citep{2025JApA...46...14G}, SNR candidates \citep{2017A&A...605A..58A,2021A&A...651A..86D,2025A&A...693A.247A}, and H{\scriptsize II} regions from the WISE catalog \citep{2014ApJS..212....1A}, respectively.} (*Fig:4.89GHz-I*)

</div>
<div id="div_fig3">

<img src="tmp_2512.15054/./fig/Parkes-Urumqi.png" alt="Fig3" width="100%"/>

**Figure 3. -** {Comparison between the Urumqi 4.8 GHz survey and the Parkes 5 GHz survey. (a) Pixel-by-pixel comparison of the observed brightness temperatures of the two survey data sets within $10^{\circ}<\ell <30^{\circ}$. The red dashed line indicates the equality between the two data sets. (b) Histogram of the brightness temperature differences between the two surveys. The vertical dashed line denotes the median value of 0.028 K. (c) Brightness temperature difference as a function of the Galactic latitude for $\ell =11\rlap{.}^{\circ}65$. The red dashed curve represents the polynomial fit to the observed distribution.} (*Fig:urmqi-parkes*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.15054"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Stellar Morphology of Optically Dark or Faint Galaxies at $z>3$ with JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.15127-b31b1b.svg)](https://arxiv.org/abs/2512.15127)<mark>Appeared on: 2025-12-18</mark> - 

</div>
<div id="authors">

A. Ganguly, et al. -- incl., <mark>S. Bosman</mark>

</div>
<div id="abstract">

**Abstract:** The sensitivity and resolution of the James Webb Space Telescope (JWST) offer an unprecedented view of optically dark or faint galaxies (OFGs), previously missed by the Hubble Space Telescope (HST). They are candidates for massive, heavily dust-obscured star-forming galaxies (SFGs) that substantially contribute to the cosmic star formation rate density at $z > 3$ . To understand what drives their high dust attenuation and how they fit into early universe galaxy evolution, we analyse the stellar morphology of 65 OFGs (selected from a parent sample of 1892 SFGs at $3<z<4$ ) using JWST/NIRCam F444W imaging from the PRIMER and CEERS legacy fields. We investigate correlations between dust attenuation ( $A_{v}$ ) and key galaxy properties, including stellar mass, size, and orientation, and compare scaling relations between OFGs and typical SFGs.We find that OFGs are $\sim$ 8--9 times more massive (median $\log M_\star/M_\odot=10.31^{+0.36}_{-0.42}$ ) and $\sim4$ times more dust attenuated ( ${A_{v,\mathrm{med}}=2.67 ^{+0.80}_{-0.96}}$ mag) than the parent sample. Structurally, OFGs resemble parent SFGs with median $R_e \sim 1$ kpc and median $\Sigma_{\mathrm{R_e}}\sim10^9,\mathrm{M_\odot/{kpc}^2}$ . At $z>3$ , OFGs may be slightly rounder on average as they prefer a higher median $q$ ( $q \sim 0.60^{+0.21}_{-0.20}$ ) than the parent SFGs ( $q \sim 0.47^{+0.22}_{-0.16}$ ), where, $q = b/a$ , b: semi-minor axis, a: semi-major axis. While $A_v$ is strongly correlated with stellar mass, it does not show significant dependence on stellar mass-normalised effective radius ( $R'_e$ ) and stellar mass surface density ( $\Sigma'_\mathrm{{R_e}}$ ), Sérsic index ( $n$ ), axis ratio ( $q$ ), or star formation rate surface density ( $\Sigma_{\mathrm{SFR}}$ ).  The mass-size and mass-surface density relations place OFGs as a higher-mass extension of SFGs, with no smoking-gun proof of evolutionary differences between them.Taken together, these results suggest that OFGs are heavily dust-obscured primarily due to their high stellar masses, which facilitates significant dust production and retention, with older stellar populations likely contributing as well. Although some OFGs exhibit high $\Sigma_\mathrm{{R_e}}$ and occupy regions of the mass-size plane similar to quiescent galaxies (QGs), the overall sample is not representative of this. Their current structures resemble typical SFGs, with no concrete signs of rapid compaction. The diversity in their physical properties indicates that OFGs span a range of evolutionary states with few showing reduced star formation, while most remain actively star-forming.

</div>

<div id="div_fig1">

<img src="tmp_2512.15127/./images/av_properties.png" alt="Fig10" width="100%"/>

**Figure 10. -** Comparison of stellar mass, stellar mass-scaled $R_{e}$($R'_{e}$, scaled to $\log M_{\star}/M_{\odot}$= 10), $q$, stellar mass-scaled $\Sigma_\mathrm{R_e}$($\Sigma'_{R_e}$, scaled to $\log M_{\star}/M_{\odot}$= 10), and star formation rate surface densities ($\Sigma_\mathrm{SFR}$) with $A_v$ of OFGs and the parent sample at $ 3 \leqslant z < 4$. Sliding medians for the two galaxy types are shown in the legend, with bold diamonds for OFGs and bold circles for parent SFGs. The error bars represent the uncertainty of the medians, and bins with fewer than ten galaxies are not displayed. For Figure \ref{fig:overall_figure_dust}b and \ref{fig:overall_figure_dust}d , the $R'_{e}$ values are $R_{e}$ values scaled to a stellar mass of $10^{10} M_{\odot}$. (*fig:overall_figure_dust*)

</div>
<div id="div_fig2">

<img src="tmp_2512.15127/./images/sigma-re.png" alt="Fig3" width="100%"/>

**Figure 3. -** Redshift-binned effective stellar surface mass density $\Sigma_\mathrm{R_{e}}$ vs. stellar mass. Grey points represent SFGs, while red points represent OFGs. The upward pointing arrows represent the same sources that lie below the resolution limit in F444W as in Figure \ref{fig:combined-mass-size}. The purple solid lines indicate the $\Sigma_\mathrm{R_{e}}$-mass relation of QGs at $2 < z < 3$, with the dashed purple lines showing the intrinsic scatter for this population, as derived from \cite{Barro2017}. The green solid and dashed lines represent the redshift-extrapolated best-fit lines and intrinsic scatter for SFGs from the same study. Black lines denote the $\Sigma_\mathrm{R_{e}}$-mass relation for the parent SFG sample at the indicated redshifts (see legend). The red line represents the $\Sigma_\mathrm{R_{e}}$-mass relation for OFGs ($3 < z < 4$), with the shaded light red and grey regions showing the $1\sigma$ scatter of the best-fit relations for OFGs and parent sample, respectively. (*fig:sigma_re*)

</div>
<div id="div_fig3">

<img src="tmp_2512.15127/./mass-size.png" alt="Fig9.1" width="50%"/><img src="tmp_2512.15127/./images/mass_size_black.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** _Left panel_: Rest-frame optical mass-size relation for parent SFGs (grey points and black lines) and OFGs (red) for $3 < z < 4  (F444W)$. The down-pointing arrows represent sources with $R_e$ smaller than the PSF HWHM or the resolution limit in F444W. These arrows indicate that the resolution limit is treated as an upper limit on their size measurements. The teal lines represent the mass-size relation at $3 < z < 4  (F444W)$ obtained from JWST Rest-frame UV/Optical sizes \citep{allen2024galaxysizemassbuildup}, while the orange lines denote those from \cite{Varadaraj2024} at $z = 3  (F356W)$ for reference. The purple lines indicate the mass-size relation of quiescent galaxies (QGs) at $z< 3.3$\citep{ito2024sizestellarmass}. Dashed lines show the 1$\sigma$ scatter of the lines from the literature. The shaded light red and grey regions show the $1\sigma$ scatter of the best-fit relations of OFGs and parent SFGs, respectively. The light green line highlights the resolution limit of the NIRCam PSF in the F444W band. _Right panel_: Here are sample RGB images from JWST/NIRCam (F115W, F277W, F444W) that effectively illustrate the distinctions between parent SFGs and OFGs. The mass-size relation derived for the parent sample, OFGs and QGs is also underlaid for comparison. Notably, the differences in morphology and color are striking across these categories. OFGs exhibit significantly redder colors compared to the parent sample. The parent sample displays a diverse range of morphologies, spanning from compact to more extended structures. In contrast, OFGs are characterised by predominantly round or ellipsoidal shapes.  (*fig:combined-mass-size*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.15127"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The impact of bars on the properties of H ${\Large **II**}$ regions in the TIMER survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.15637-b31b1b.svg)](https://arxiv.org/abs/2512.15637)<mark>Appeared on: 2025-12-18</mark> -  _Published in MNRAS_

</div>
<div id="authors">

L. Sánchez-Menguiano, et al. -- incl., <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** In this study we perform a comparative analysis of the properties of the HII regions located in different areas of barred galaxies, with the aim of investigating the impact of bars on the physical properties of the ionised gas. Based on integral field spectroscopy data for 17 barred galaxies covering approximately the central $6\times6$ kpc, we detect a total of 2200 $\ion{H}{ii}$ regions, of which 331 are located within the nuclear disc (also known as circumnuclear regions),  661 in the bar region, and 1208 in the disc. Among the physical properties of the $\ion{H}{ii}$ regions, we explore the O/H and N/O abundances, H $\alpha$ luminosity, dust extinction, electron density, and H $\alpha$ equivalent width. We find clear differences in the properties of the $\ion{H}{ii}$ regions between the nuclear disc, the bar and the disc, that could be explained by an enhancement in the molecular gas concentration in the central parts driven by bar-induced gas flows. As this gas is channelled towards the galaxy centre, the most extreme values in the analysed properties are found for the circumnuclear $\ion{H}{ii}$ regions. Unlike the bar strength, galaxy mass does seem to affect the properties of the $\ion{H}{ii}$ regions, with massive galaxies presenting higher values in most of the properties, possibly due to the increased amount of gas in these systems. This study provides evidence that the bar-driven redistribution of material within the galaxy inner parts causes significant differences in the $\ion{H}{ii}$ region properties depending on their location within the galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2512.15637/./histograms1_v3.png" alt="Fig4" width="100%"/>

**Figure 4. -** Comparison of the distributions of oxygen abundances (_ top_), N/O abundances (_ middle_), and L(H$\alpha$) (_ bottom_) for different populations of $\ion${H}{ii} regions: disc (green) versus bar (orange) regions (_ left_), disc (green) versus circumnuclear (pink) regions (_ middle_), and regions in the disc inside (purple) versus outside (grey) the bar radius (_ right_). The distribution for the bar regions is also represented in right panels with unfilled black histograms. In the top left corner of first panels we include the typical (median) errorbar of each parameter, derived by propagating the errors in the involved emission line intensities (the intrinsic error in the used calibrations is not included for O/H and N/O abundances). In the bottom right corner of the panels the Mann-Whitney U test p-value for each pair of compared distributions is shown, and for those which have values below 5\%(i.e. presenting statistically significant differences), the Cliff's $\delta$ is also provided as a measure of the magnitude of the differences. These are categorised as negligible (N), small (S), medium (M) and large (L) differences (see Sec. \ref{sec:results} for details on these statistics). (*fig:hists1*)

</div>
<div id="div_fig2">

<img src="tmp_2512.15637/./pyHIIEXTRACTOR_plot.png" alt="Fig1" width="100%"/>

**Figure 1. -** Modelling of the $\ion${H}{ii} regions by {\ttfamily PYHIIEXTRACTOR} for the galaxy NGC 4981. We show the observed H$\alpha$ map (top panel), the derived model of the candidate $\ion${H}{ii} regions and the DIG together (middle-top panel) and separately ($\ion${H}{ii} regions, middle-bottom panel; DIG, bottom panel). Orange circle and red ellipse in top panel delimit the nuclear disc and bar regions, respectively. (*fig:HIIregions*)

</div>
<div id="div_fig3">

<img src="tmp_2512.15637/./histograms2_v3.png" alt="Fig5" width="100%"/>

**Figure 5. -** Same as Fig. \ref{fig:hists1} but for the dust extinction (_ top_), the $[$\ion${S}{ii}]\lambda6717/[$\ion${S}{ii}]\lambda6731$ line ratio as a proxy for the electron density (_ middle_), and the H$\alpha$ equivalent width (_ bottom_). See caption above for more details. (*fig:hists2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.15637"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

134  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
